<font size="6"> Tutorial I </font>

<font size="9"> Reading in data </font>

This first tutorial explains how LIS output can be read in.

**Table of contents**<a id='toc0_'></a>    
- [1. LIS model output](#toc1_)    
- [2. Innovations, increments and spread](#toc2_)    
- [3. Satellite observations](#toc3_)    
- [4. Ancillary data](#toc4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

Always start your script or notebook by enabling `pylis`. Replace the path in the following block by the foler in which your version of `pylis` is located.

In [18]:
import sys
sys.path.append("/dodrio/scratch/projects/2022_200/project_output/rsda/vsc34049")

# <a id='toc1_'></a>[1. LIS model output](#toc0_)

The `pylis.readers` module contains routines that enable reading in LIS output. The `lis_cube` function reads in a data cube of a model variable (3- or 4-dimensional). The inputs are:

| Input       | Default | Explanation   |
|:----------|:----------|:----------|
| `lis_dir` | <!-- --> | Where is the LIS output stored? This is the path provided in the `lis.config` file, i.e., don't include `"SURFACEMODEL"`.   |
| `lis_input_file` | <!-- --> | The LDT output with the LIS domain. We need this for the latitude and longitude (these variables are masked over water in the LIS output). |
| `var` | <!-- --> | The variable for which to read in the data cube. Make sure to use the correct suffix (`_inst` or `_tavg`). |
| `start` | <!-- --> | First date of the data cube (format `DD/MM/YYYY`). |
| `end` | <!-- --> | Last date of the data cube (format `DD/MM/YYYY`). |
| `subfolder` | `"SURFACEMODEL"` | Only change if your data are not stored in the `"SURFACEMODEL"` folder (e.g., for VOD or irrigation). |
| `d` | `"01"` | Only change if your are using nested domains. |
| `freq` | `"1D"` | Only change if you don't have daily outputs (e.g., `"1H"` for hourly outputs). |
| `date_shift` | `False` | Whether to shift the LIS output date with `freq`. Recommended for variables with `_tavg` suffix, as it is the average since the previous output (default `False` to be consistent with old behavior, but a warning is raised when shifting is not applied with averaged output). |

As an example, we can read in a data cube of soil moisture.

In [27]:
from pylis import readers

# we will need the lis_input file to obtain the latitude and longitude
lis_input_file = "/dodrio/scratch/projects/2022_200/project_output/rsda/vsc34049/awu/lis_input_files/lis_input_CONUS_notile_noirr_NoahMP4.0.1.nc"

# read in soil moisture data cube over US (hourly data)
dc_sm = readers.lis_cube(
    lis_dir = "/dodrio/scratch/projects/2022_200/project_output/rsda/vsc34049/awu/notile_noirr/OUT_det_CONUS",
    lis_input_file = lis_input_file,
    var = "SoilMoist_tavg",
    start = "01/01/2020",
    end = "31/01/2020",
    freq = "1H",
    date_shift = True
)

  0%|          | 0/721 [00:00<?, ?it/s]

The resulting object is an `xr.DataArray` with in this case 4 dimensions: time, soil layer, and x- and y-direction of the grid. For a regular latitude-longitude grid such as this one, `x` and `y` will have a one-to-one correspondence with the longitude and latitude coordinates respectively. For non-linear grids (e.g., Lambert conformal) this will not be the case, the `lat` and `lon` coordinates themselves are then two-dimensional on the `xy`-grid themselves.

In [30]:
dc_sm

<xarray.DataArray (time: 721, layer: 4, x: 99, y: 233)>
array([[[[       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         ...,
         [0.32327461, 0.32563323, 0.32579201, ...,        nan,
          0.2523891 , 0.24915573],
         [0.31529155, 0.31999439, 0.31350398, ...,        nan,
                 nan,        nan],
         [0.30165663, 0.29273632,        nan, ..., 0.21526513,
                 nan,        nan]],

        [[       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
...
         [0.28625286, 0.28980643, 0.28522253, ...,        nan,
          0.25050709, 0.24839184],
         [0.28869605, 0.2836017 , 0.28589562, ...,        nan,
                 nan,        nan],
         [0.28773913, 0.27948284,        nan, ..., 0.21175064,
                 nan,        nan]],

        [[       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         ...,
         [0.29162526, 0.29521263, 0.28741559, ...,        nan,
          0.25777695, 0.25532672],
         [0.29257256, 0.2850334 , 0.28984731, ...,        nan,
                 nan,        nan],
         [0.29130983, 0.28080344,        nan, ..., 0.21905822,
                 nan,        nan]]]])
Coordinates:
    lon      (x, y) float32 -124.9 -124.6 -124.4 -124.1 ... -67.38 -67.12 -66.88
    lat      (x, y) float32 24.88 24.88 24.88 24.88 ... 49.38 49.38 49.38 49.38
  * layer    (layer) int64 1 2 3 4
  * time     (time) datetime64[ns] 2020-01-01 2020-01-01T01:00:00 ... 2020-01-31
Dimensions without coordinates: x, y
Attributes:
    description:  LIS model output
    variable:     SoilMoist_tavg

A handy function to compute the root-zone soil moisture is available in the `pylis.help` module:

In [14]:
from pylis.help import root_zone

dc_rzsm = root_zone(dc_sm)

Default depths of the layers are for the Noah-MP model, but any list can be provided through the optional `weights` argument:

In [15]:
?root_zone

Signature: root_zone(dc, weights=[0.1, 0.3, 0.6])
Docstring:
Compute the root-zone (soil moisture/temperature) based on a weighted average of the layers.
Default weights work for Noah-MP output.
File:      /dodrio/scratch/projects/2022_200/project_output/rsda/vsc34049/pylis/help.py
Type:      function

Next time, it is much quicker to read in the data if you store this `xr.DataArray` as a netcdf file. First give the variable a name since the `to_netcdf` function works on `xr.DataSets` rather than `xr.DataArrays`.

In [ ]:
dc_sm.to_dataset(name = "SoilMoisture").to_netcdf("/path/to/folder/dc_sm.nc")

In [ ]:
import xarray as xr

# faster next time we need to use the data
dc_sm = xr.open_dataset("/path/to/folder/dc_sm.nc").SoilMoisture

# <a id='toc2_'></a>[2. Innovations, increments and spread](#toc0_)
If you ran a DA experiment, you will also have an `EnKF` folder (containing innovations, increments, spread) alsongside the `SURFACEMODEL` folder. You can use the `innov_cube` and `incr_cube` functions to read in innovations and increments.
For `innov_cube`, the inputs are:
| Input       | Default | Explanation   |
|:----------|:----------|:----------|
| `lis_dir` | <!-- --> | Where is the LIS output stored? This is the path provided in the `lis.config` file, i.e., don't include `"SURFACEMODEL"`.   |
| `lis_input_file` | <!-- --> | The LDT output with the LIS domain. We need this for the latitude and longitude (these variables are masked over water in the LIS output). |
| `start` | <!-- --> | First date of the data cube (format `DD/MM/YYYY`). |
| `end` | <!-- --> | Last date of the data cube (format `DD/MM/YYYY`). |
| `var` |`"innov"` | The variable for which to read in the data cube. `innov` for innovations; `ninnov` for normalized innovations. |
| `subfolder` | `"EnKF"` | Only change if your data are not stored in the `"EnKF"` folder. |
| `a` | `"01"` | Only change if your are performing a multi-sensor DA. |
| `d` | `"01"` | Only change if your are using nested domains. |
| `freq` | `None` | The temporal frequency to which the innovations should be *resampled* by averaging, e.g. `"1D"` to have daily average innovations (default `None`, i.e., don't resample). |

Example reading the normalized innovations:

In [16]:
from pylis import readers

# we will need the lis_input file to obtain the latitude and longitude
lis_input_file = "/dodrio/scratch/projects/2022_200/project_output/rsda/vsc34049/nu-wrf/OL_113_401/lis_input.d02.nc"

sm_ninnov = readers.innov_cube(
    lis_dir = "/dodrio/scratch/projects/2022_200/project_output/rsda/vsc34049/nu-wrf/smap_da_offline_monthly/d02/OUTPUT_with_extra_masking",
    lis_input_file = lis_input_file,
    var = "ninnov",
    start = "01/01/2020",
    end = "31/01/2020",
)

Constructing innovation cube ...


100%|█████████████████████████████████████| 15602/15602 [06:15<00:00, 41.59it/s]


In [18]:
dc_ninnov

<xarray.DataArray (time: 142, x: 159, y: 261)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
    lon      (x, y) float32 -120.8 -120.6 -120.5 -120.3 ... -67.91 -67.69 -67.46
    lat      (x, y) float32 22.8 22.84 22.87 22.91 ... 47.4 47.35 47.3 47.25
  * time     (time) datetime64[ns] 2020-01-01T11:15:00 ... 2020-01-30T23:15:00
Dimensions without coordinates: x, y
Attributes:
    description:  LIS innovations
    variable:     ninnov_01

Behavior is similar for `incr_cube`. Here, the `var` argument relates to the model state (e.g., `"Soil Moisture"`, `"LAI"`, ...) as defined in the LIS increment files. An additional argument `layers` expects `None` for a variable without layers such as LAI, and a list (e.g., `[1]` or `[1,2,3,4]`) for a variable with layer such as soil moisture. Increments that are exactly zero are assigned a missing value: they correspond to times and locations without available observations.

Ensemble spread can be read in using the `spread_cube` function. Inputs are the same as for `incr_cube`. Note that the `freq` argument in this case refers to the model output frequency (as in `lis_cube`), not the desired temporal frequency after resampling. It defaults to `"1D"`.

# <a id='toc3_'></a>[3. Satellite observations](#toc0_)

After completing a DA run in LIS, satellite observations are stored in binary files in the `DAOBS` subfolder. You can read them in via `pylis` using the `obs_cube` function. The inputs are:
* `lis_dir`: where is the LIS output stored? This is the path provided in the `lis.config` file, i.e., don't include `"DAOBS"`;
* `lis_input_file`: the LDT output with the LIS domain. We need this for the latitude and longitude (these variables are masked over water in the LIS output);
* `start`: first date of the data cube;
* `end`: last date of the data cube;
* `rescaled`: if a rescaling is performed (e.g., CDF matching), you can choose to read in the original or rescaled observations (default `False`);
* `a`: only change if you perform a multi-sensor DA (default `"01"`);
* `d`: only change if your are using nested domains (default `"01"`);
* `freq`: the temporal frequency to which observations should be resampled by averaging (default `None`, i.e., don't resample).

In [23]:
from pylis import readers

# we will need the lis_input file to obtain the latitude and longitude
lis_input_file = "/dodrio/scratch/projects/2022_200/project_output/rsda/vsc34049/nu-wrf/OL_113_401/lis_input.d02.nc"

dc_obs = readers.obs_cube(
    lis_dir = "/dodrio/scratch/projects/2022_200/project_output/rsda/vsc34049/nu-wrf/smap_da_offline_monthly/d02/OUTPUT_with_extra_masking",
    lis_input_file = lis_input_file,
    start = "01/01/2020",
    end = "31/01/2020",
    rescaled = True
)

Counting the number of observations ...
Constructing observation cube ...


100%|█████████████████████████████████████| 15602/15602 [07:08<00:00, 36.45it/s]


In [24]:
dc_obs

<xarray.DataArray (time: 142, x: 159, y: 261)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
    lon      (x, y) float32 -120.8 -120.6 -120.5 -120.3 ... -67.91 -67.69 -67.46
    lat      (x, y) float32 22.8 22.84 22.87 22.91 ... 47.4 47.35 47.3 47.25
  * time     (time) datetime64[ns] 2020-01-01T11:15:00 ... 2020-01-30T23:15:00
Dimensions without coordinates: x, y
Attributes:
    description:  Observations obtained form binaray DAOBS files

You can easily count the total number of observations in time and space:

In [31]:
import numpy as np

np.isfinite(dc_obs).sum(dim = ("time", "x", "y")).values

array(148458)

# <a id='toc4_'></a>[4. Ancillary data](#toc0_)
There are some more functions in `pylis.readers` to read out the landmask and landcover from the LIS input file.
* `landflag` takes `lis_input_file` as input and returns a binary `xr.DataArray`;
* `landcover` takes `lis_input_file` as input. It either returns a 2D array of majority landcover classes (if `majority = True`) or a 3D array with fraction of each landcover class (if `majority = False`).

In [34]:
from pylis import readers

# we will need the lis_input file to obtain the latitude and longitude
lis_input_file = "/dodrio/scratch/projects/2022_200/project_output/rsda/vsc34049/awu/lis_input_files/lis_input_CONUS_notile_noirr_NoahMP4.0.1.nc"

lc = readers.landcover(lis_input_file, majority = True)
lc

<xarray.DataArray (x: 99, y: 233)>
array([['Water', 'Water', 'Water', ..., 'Water', 'Water', 'Water'],
       ['Water', 'Water', 'Water', ..., 'Water', 'Water', 'Water'],
       ['Water', 'Water', 'Water', ..., 'Water', 'Water', 'Water'],
       ...,
       ['Evergreen Needleleaf Forest', 'Evergreen Needleleaf Forest',
        'Evergreen Needleleaf Forest', ..., 'Water', 'Mixed Forests',
        'Mixed Forests'],
       ['Evergreen Needleleaf Forest', 'Evergreen Needleleaf Forest',
        'Evergreen Needleleaf Forest', ..., 'Water', 'Water', 'Water'],
       ['Evergreen Needleleaf Forest', 'Evergreen Needleleaf Forest',
        'Water', ..., 'Mixed Forests', 'Water', 'Water']], dtype='<U100')
Coordinates:
    lon      (x, y) float32 -124.9 -124.6 -124.4 -124.1 ... -67.38 -67.12 -66.88
    lat      (x, y) float32 24.88 24.88 24.88 24.88 ... 49.38 49.38 49.38 49.38
Dimensions without coordinates: x, y